## Evaluation with Standard Benchmarks: Coherence
### Using evaluation tool for word embeddings

Here, we apply standard benchmarks on coherence on w2v and debiased w2v.

In [1]:
# Subset of GoogleNews-vectors:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

import benchmarks as benchmarks
from benchmarks.wordsim.wordsim import Wordsim

# Small w2vNEWS set

## 1: original word embeddings on RG & WS

Sources:

#### RG: H. Rubenstein and J. B. Goodenough. Contextual correlates of synonymy. Communications of the ACM, 8(10):627–633, 1965.

####  WS: L. Finkelstein, E. Gabrilovich, Y. Matias, E. Rivlin, Z. Solan, G. Wolfman, and E. Ruppin. Placing search in context: The concept  revisited. In WWW. ACM, 2001.


In [3]:
# Load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


### Wordsim benchmarks
Code adapted from source 

#### embedding-evaluation: https://github.com/k-kawakami/embedding-evaluation

In [4]:
wordsim = Wordsim("en")
word2vec = wordsim.load_vector('./embeddings/w2v_gnews_small.txt')
result_original = wordsim.evaluate(word2vec)
wordsim.pprint(result_original)

load_vector
loading vector...
b'in'
b'for'
b'that'
b'is'
b'on'
b'with'
b'said'
b'was'
b'the'
b'at'
b'not'
b'as'
b'it'
b'be'
b'from'
b'by'
b'are'
b'have'
b'he'
b'will'
b'has'
b'his'
b'an'
b'this'
b'or'
b'their'
b'who'
b'they'
b'but'
b'had'
b'year'
b'were'
b'we'
b'more'
b'up'
b'been'
b'you'
b'its'
b'one'
b'about'
b'would'
b'which'
b'out'
b'can'
b'all'
b'also'
b'two'
b'after'
b'first'
b'do'
b'time'
b'than'
b'when'
b'over'
b'last'
b'new'
b'other'
b'her'
b'people'
b'into'
b'our'
b'there'
b'she'
b'could'
b'just'
b'years'
b'some'
b'three'
b'million'
b'them'
b'what'
b'so'
b'no'
b'like'
b'if'
b'only'
b'percent'
b'get'
b'did'
b'him'
b'game'
b'back'
b'because'
b'now'
b'before'
b'company'
b'any'
b'team'
b'against'
b'off'
b'most'
b'made'
b'through'
b'make'
b'second'
b'state'
b'well'
b'day'
b'season'
b'says'
b'week'
b'where'
b'while'
b'down'
b'being'
b'government'
b'your'
b'home'
b'going'
b'my'
b'good'
b'should'
b'many'
b'way'
b'those'
b'four'
b'during'
b'such'
b'may'
b'very'
b'how'
b'since'
b'work'

b'television'
b'rules'
b'rights'
b'review'
b'quite'
b'regular'
b'served'
b'kept'
b'marketing'
b'created'
b'heart'
b'wrote'
b'prison'
b'instead'
b'concerns'
b'offering'
b'returned'
b'benefit'
b'designed'
b'approved'
b'foot'
b'search'
b'miles'
b'planned'
b'driving'
b'entire'
b'providing'
b'host'
b'ask'
b'design'
b'hearing'
b'save'
b'looked'
b'travel'
b'successful'
b'beginning'
b'movie'
b'adding'
b'forced'
b'toward'
b'ensure'
b'passed'
b'unit'
b'happen'
b'mean'
b'shooting'
b'gone'
b'winner'
b'hopes'
b'understand'
b'planning'
b'trip'
b'raise'
b'owner'
b'popular'
b'completed'
b'ways'
b'attention'
b'feet'
b'assets'
b'traffic'
b'environment'
b'interview'
b'consumers'
b'hands'
b'contact'
b'cars'
b'dead'
b'features'
b'challenge'
b'particularly'
b'basis'
b'violence'
b'stand'
b'consumer'
b'sense'
b'attorney'
b'driver'
b'highest'
b'posted'
b'finish'
b'limited'
b'source'
b'serve'
b'spot'
b'advantage'
b'scoring'
b'words'
b'applications'
b'reduce'
b'white'
b'appeared'
b'itself'
b'fuel'
b'simply'
b'fi

b'volume'
b'effects'
b'tests'
b'interests'
b'occurred'
b'bond'
b'introduced'
b'sport'
b'spokeswoman'
b'behavior'
b'defeat'
b'treated'
b'direction'
b'bigger'
b'crew'
b'allegedly'
b'uses'
b'mortgage'
b'miss'
b'catch'
b'appearance'
b'welcome'
b'experienced'
b'ideas'
b'negotiations'
b'authority'
b'slightly'
b'urged'
b'hole'
b'reform'
b'achieve'
b'assistance'
b'delivered'
b'compete'
b'link'
b'choose'
b'possibility'
b'ruled'
b'interesting'
b'seats'
b'threw'
b'drove'
b'immediate'
b'investments'
b'payments'
b'northern'
b'relations'
b'license'
b'discovered'
b'prove'
b'registered'
b'perform'
b'governments'
b'debut'
b'researchers'
b'accounts'
b'limit'
b'eyes'
b'deaths'
b'neighborhood'
b'handed'
b'veteran'
b'alternative'
b'basic'
b'manage'
b'tools'
b'negative'
b'secretary'
b'kill'
b'receiving'
b'collection'
b'coaches'
b'returns'
b'surgery'
b'gun'
b'replace'
b'fan'
b'institutions'
b'personnel'
b'golf'
b'happens'
b'surprise'
b'changing'
b'kick'
b'ceremony'
b'west'
b'resolution'
b'broken'
b'plenty'
b

b'poverty'
b'properly'
b'somebody'
b'canceled'
b'sounds'
b'sick'
b'saves'
b'lucky'
b'folks'
b'defend'
b'lights'
b'minority'
b'posting'
b'drilling'
b'differences'
b'regularly'
b'roster'
b'bankruptcy'
b'participation'
b'interim'
b'controls'
b'competitors'
b'stuck'
b'delivering'
b'viewers'
b'agreements'
b'subscribers'
b'smoke'
b'compliance'
b'manner'
b'trucks'
b'yes'
b'mistakes'
b'formal'
b'investor'
b'speculation'
b'qualifying'
b'attorneys'
b'finds'
b'quiet'
b'cleared'
b'bag'
b'explosion'
b'taxpayers'
b'guide'
b'occur'
b'encouraging'
b'highway'
b'sleep'
b'recover'
b'interviews'
b'satellite'
b'entirely'
b'dry'
b'papers'
b'soldier'
b'covers'
b'exhibition'
b'blast'
b'capture'
b'function'
b'availability'
b'lending'
b'peak'
b'wild'
b'facts'
b'procedures'
b'lap'
b'awareness'
b'pursue'
b'edition'
b'studio'
b'correct'
b'inch'
b'switch'
b'participating'
b'tend'
b'champions'
b'lay'
b'purposes'
b'departments'
b'ranking'
b'addressed'
b'perspective'
b'components'
b'seized'
b'clothing'
b'drawing'
b're

b'affecting'
b'rugby'
b'forest'
b'cloud'
b'contractor'
b'chest'
b'signature'
b'uniform'
b'accidents'
b'intervention'
b'tougher'
b'teenager'
b'gambling'
b'mortgages'
b'standings'
b'evaluation'
b'planet'
b'dressed'
b'underground'
b'exact'
b'conservation'
b'midfielder'
b'depend'
b'replacing'
b'headline'
b'questioning'
b'athlete'
b'wide_range'
b'comparable'
b'accomplished'
b'ordinance'
b'feared'
b'discussing'
b'riders'
b'copper'
b'unexpected'
b'administrators'
b'favorable'
b'painting'
b'comply'
b'headquartered'
b'gross'
b'picks'
b'diplomatic'
b'lighting'
b'tobacco'
b'feelings'
b'pays'
b'documentary'
b'addresses'
b'senators'
b'linebacker'
b'cleaning'
b'withdrawal'
b'brilliant'
b'custom'
b'priorities'
b'economist'
b'waited'
b'furniture'
b'sub'
b'turnover'
b'costly'
b'height'
b'volumes'
b'till'
b'grab'
b'completing'
b'sample'
b'equipped'
b'delegates'
b'favorites'
b'shortage'
b'publisher'
b'estate'
b'hitter'
b'globe'
b'evaluate'
b'sits'
b'venture'
b'shoppers'
b'designated'
b'tying'
b'signals'


...

b'pinpointed'
b'bio_email'
b'helpline'
b'whips'
b'hiccup'
b'scuttle'
b'moody'
b'shooting_spree'
b'substantiated'
b'motherboard'
b'sew'
b'relished'
b'vocation'
b'hairy'
b'impeding'
b'enemy_combatants'
b'rudimentary'
b'll'
b'radioed'
b'bliss'
b'exaggerate'
b'statins'
b'distinguishes'
b'wheeling'
b'lavender'
b'jihadists'
b'zipped'
b'bloodbath'
b'expeditions'
b'humankind'
b'strategic_alliances'
b'pods'
b'sculpted'
b'multitasking'
b'electrocuted'
b'ratchet'
b'illiteracy'
b'austere'
b'preachers'
b'directional'
b'co_ordinate'
b'potentials'
b'hooded'
b'bargaining_table'
b'rejoining'
b'arid'
b'vilified'
b'smacks'
b'kilowatt_hours'
b'eighty'
b'cinematography'
b'axle'
b'hilltop'
b'semifinalists'
b'interconnection'
b'shutters'
b'bowel'
b'kayaking'
b'estranged_husband'
b'recessionary'
b'reappear'
b'mummy'
b'nontraditional'
b'polymers'
b'deferral'
b'goalscorer'
b'vector'
b'summarize'
b'tuneup'
b'flailing'
b'avoidable'
b'whine'
b'purposeful'
b'coaxed'
b'puberty'
b'scrawled'
b'reappointed'
b'unsavory'

b'cougar'
b'steelhead'
b'confiscation'
b'deviate'
b'net_neutrality'
b'straddles'
b'avocado'
b'delisted'
b'bacterium'
b'subcontinent'
b'uninformed'
b'loudspeaker'
b'defibrillator'
b'pardons'
b'supply_disruptions'
b'recollections'
b'jewelers'
b'demoralized'
b'carbon_capture'
b'nerve_cells'
b'spurts'
b'aptitude'
b'brokering'
b'spear'
b'midget'
b'bereaved'
b'leadoff_homer'
b'uber'
b'spoilers'
b'contours'
b'reciting'
b'ferried'
b'decreed'
b'zip_code'
b'legitimize'
b'summarizes'
b'colds'
b'massed'
b'penitentiary'
b'flanks'
b'impediments'
b'wink'
b'elastic'
b'absenteeism'
b'conclave'
b'mascots'
b'showering'
b'environs'
b'firming'
b'dress_rehearsal'
b'downsized'
b'swine_flu_vaccine'
b'remain_vigilant'
b'precincts_reporting'
b'reaffirming'
b'worldview'
b'replicating'
b'bunches'
b'aunts'
b'brooding'
b'splendor'
b'skits'
b'sought_refuge'
b'dud'
b'steelmakers'
b'slugging'
b'blank_check'
b'brokerage_firms'
b'amenity'
b'electrodes'
b'awaken'
b'prejudiced'
b'freaks'
b'bobbled'
b'remorseful'
b'enforce

b'flat_tire'
b'enchanting'
b'choppers'
b'cobalt'
b'humiliate'
b'infuriating'
b'octopus'
b'realtor'
b'smelly'
b'unending'
b'biographer'
b'mental_toughness'
b'purged'
b'machinations'
b'purporting'
b'ensuing_possession'
b'trash_bin'
b'coastal_waters'
b'languish'
b'cylinder_engine'
b'von'
b'bonnet'
b'misdemeanor_counts'
b'existential'
b'vagina'
b'dermatologist'
b'dilemmas'
b'sports_memorabilia'
b'mains'
b'glazed'
b'hormonal'
b'checkup'
b'transparently'
b'nationalize'
b'prejudicial'
b'saturated_fat'
b'disintegrated'
b'zooming'
b'jacked'
b'brainstorming'
b'entitles'
b'negatively_affected'
b'finalization'
b'ascertained'
b'camcorders'
b'pasted'
b'gaping'
b'considerate'
b'confessional'
b'teen_pregnancy'
b'anecdotal_evidence'
b'quizzes'
b'domino_effect'
b'shudder'
b'headsets'
b'soloists'
b'reinvestment'
b'cryptic'
b'airman'
b'selector'
b'dermatology'
b'flirt'
b'downwards'
b'corrects'
b'adversely'
b'biometrics'
b'bullshit'
b'constrain'
b'uncanny'
b'seep'
b'sweetest'
b'straddling'
b'mono'
b'heartl

b'slugged'
b'profess'
b'encircled'
b'decommissioned'
b'doves'
b'registries'
b'viewpoints_race'
b'fiddling'
b'silliness'
b'mistaken_identity'
b'commonsense'
b'heavily_populated'
b'cachet'
b'costliest'
b'toying'
b'embedding'
b'ghostly'
b'artisan'
b'distilled'
b'intestine'
b'accrual'
b'harsh_criticism'
b'unaffiliated'
b'dyes'
b'meted'
b'sediments'
b'navigator'
b'incurable'
b'eloquently'
b'ethereal'
b'blockades'
b'watercolors'
b'reelected'
b'motherland'
b'creditable'
b'risk_averse'
b'mechanically'
b'energizing'
b'fifties'
b'unassailable'
b'abandons'
b'virginity'
b'axing'
b'rollercoaster'
b'pricier'
b'asterisk'
b'corals'
b'contemplates'
b'senatorial'
b'parred'
b'geo'
b'intensively'
b'fazed'
b'wearable'
b'wronged'
b'anomalous'
b'opulent'
b'armistice'
b'facebook'
b'policymaking'
b'assimilate'
b'unremarkable'
b'interstates'
b'coastlines'
b'link_tags'
b'kick_returner'
b'inherits'
b'abhorrent'
b'polyester'
b'passerby'
b'aptly_named'
b'tellers'
b'consecutively'
b'airway'
b'counterpoint'
b'nomadic

b'parsley'
b'panchayat'
b'insecticide'
b'taco'
b'chuck'
b'burgundy'
b'wobble'
b'reinvention'
b'bouncers'
b'racist_vulgar'
b'deteriorates'
b'closers'
b'delves'
b'mar'
b'implausible'
b'earthly'
b'thawing'
b'crescendo'
b'co_ordinating'
b'bane'
b'fiesta'
b'whipped_cream'
b'sawmill'
b'muffins'
b'tractor_trailers'
b'multi_disciplinary'
b'modems'
b'feverish'
b'frontrunners'
b'riser'
b'scuba_diving'
b'adoration'
b'mainstays'
b'lodged_complaint'
b'fighter_pilot'
b'junctions'
b'stabilizes'
b'puffing'
b'recycled_materials'
b'moan'
b'water_heaters'
b'liturgy'
b'undrafted'
b'functionaries'
b'winningest_coach'
b'taut'
b'knelt'
b'org'
b'spray_paint'
b'heckled'
b'retelling'
b'flinch'
b'mouthed'
b'glancing'
b'gain_traction'
b'celery'
b'steelmaker'
b'cacophony'
b'thereby_reducing'
b'concentration_camps'
b'obscene_offensive'
b'dungeon'
b'tackler'
b'leveraged_buyout'
b'poop'
b'hookup'
b'inquisitive'
b'spokespeople'
b'safeguarded'
b'stab_wound'
b'beheadings'
b'bodily_injury'
b'mod'
b'dispersal'
b'marriage_

## 2: Debiased word embeddings on RG & WS


### Step 2a: Hard debiased

In [5]:
from debiaswe.debias import debias

In [6]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
# print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]


In [7]:
debias(E, gender_specific_words, defs, equalize_pairs)
print("Saving to file...")
E.save('./embeddings/w2v_gnews_debiased_small.txt')
print("\n\nDone!\n")

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('gentlemen', 'ladies'), ('males', 'females'), ('FATHERHOOD', 'MOTHERHOOD'), ('FRATERNITY', 'SORORITY'), ('Father', 'Mother'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('dudes', 'gals'), ('KINGS', 'QUEENS'), ('dad', 'mom'), ('catholic_priest', 'nun'), ('boy', 'girl'), ('twin_brother', 'twin_sister'), ('CATHOLIC_PRIEST', 'NUN'), ('colt', 'filly'), ('COUNCILMAN', 'COUNCILWOMAN'), ('boys', 'girls'), ('King', 'Queen'), ('His', 'Her'), ('FATHERS', 'MOTHERS'), ('dads', 'moms'), ('prince', 'princess'), ('GELDING', 'MARE'), ('Spokesman', 'Spokeswoman'), ('Male', 'Female'), ('Wives', 'Husbands'), ('COLT', 'FILLY'), ('Brothers', 'Sisters'), ('Councilman', 'Councilwoman'), ('Fathers', 'Mothers'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('his', 'her'), ('grandson', 'granddaughter'), ('HE', 'SHE'), ('KING', 'QUEEN'), ('Businessman', 'Businesswoman'), ('Fella', 'Granny'), ('Testosterone', 'Estrogen'), ('HIS', 'HER'), ('Chairman', 'Chai

In [8]:
# Do benchmark for hard-debiased
wordsim = Wordsim("en")
word2vec = wordsim.load_vector('./embeddings/w2v_gnews_debiased_small.txt')
result_debiased = wordsim.evaluate(word2vec)
print("ORIGINAL")
wordsim.pprint(result_original)
print("DEBIASED")
wordsim.pprint(result_debiased)

load_vector
loading vector...
b'in'
b'for'
b'that'
b'is'
b'on'
b'with'
b'said'
b'was'
b'the'
b'at'
b'not'
b'as'
b'it'
b'be'
b'from'
b'by'
b'are'
b'have'
b'he'
b'will'
b'has'
b'his'
b'an'
b'this'
b'or'
b'their'
b'who'
b'they'
b'but'
b'had'
b'year'
b'were'
b'we'
b'more'
b'up'
b'been'
b'you'
b'its'
b'one'
b'about'
b'would'
b'which'
b'out'
b'can'
b'all'
b'also'
b'two'
b'after'
b'first'
b'do'
b'time'
b'than'
b'when'
b'over'
b'last'
b'new'
b'other'
b'her'
b'people'
b'into'
b'our'
b'there'
b'she'
b'could'
b'just'
b'years'
b'some'
b'three'
b'million'
b'them'
b'what'
b'so'
b'no'
b'like'
b'if'
b'only'
b'percent'
b'get'
b'did'
b'him'
b'game'
b'back'
b'because'
b'now'
b'before'
b'company'
b'any'
b'team'
b'against'
b'off'
b'most'
b'made'
b'through'
b'make'
b'second'
b'state'
b'well'
b'day'
b'season'
b'says'
b'week'
b'where'
b'while'
b'down'
b'being'
b'government'
b'your'
b'home'
b'going'
b'my'
b'good'
b'should'
b'many'
b'way'
b'those'
b'four'
b'during'
b'such'
b'may'
b'very'
b'how'
b'since'
b'work'

b'miles'
b'planned'
b'driving'
b'entire'
b'providing'
b'host'
b'ask'
b'design'
b'hearing'
b'save'
b'looked'
b'travel'
b'successful'
b'beginning'
b'movie'
b'adding'
b'forced'
b'toward'
b'ensure'
b'passed'
b'unit'
b'happen'
b'mean'
b'shooting'
b'gone'
b'winner'
b'hopes'
b'understand'
b'planning'
b'trip'
b'raise'
b'owner'
b'popular'
b'completed'
b'ways'
b'attention'
b'feet'
b'assets'
b'traffic'
b'environment'
b'interview'
b'consumers'
b'hands'
b'contact'
b'cars'
b'dead'
b'features'
b'challenge'
b'particularly'
b'basis'
b'violence'
b'stand'
b'consumer'
b'sense'
b'attorney'
b'driver'
b'highest'
b'posted'
b'finish'
b'limited'
b'source'
b'serve'
b'spot'
b'advantage'
b'scoring'
b'words'
b'applications'
b'reduce'
b'white'
b'appeared'
b'itself'
b'fuel'
b'simply'
b'fiscal'
b'plays'
b'account'
b'follow'
b'statements'
b'organizations'
b'pick'
b'continues'
b'session'
b'nuclear'
b'seconds'
b'daughter'
b'co'
b'sports'
b'caused'
b'injury'
b'effect'
b'selling'
b'middle'
b'cuts'
b'drop'
b'facility'
b'tru

b'dedicated'
b'tight'
b'lawyers'
b'sets'
b'linked'
b'providers'
b'investigators'
b'stands'
b'terrorism'
b'arm'
b'freedom'
b'notice'
b'sentenced'
b'coalition'
b'touch'
b'exercise'
b'settlement'
b'disaster'
b'assist'
b'electronic'
b'playoffs'
b'combination'
b'apply'
b'shared'
b'doctor'
b'comprehensive'
b'earn'
b'fill'
b'rock'
b'giant'
b'please'
b'suit'
b'owns'
b'electricity'
b'exciting'
b'winners'
b'beating'
b'protest'
b'rejected'
b'ticket'
b'farmers'
b'ride'
b'maker'
b'heading'
b'regulations'
b'models'
b'ends'
b'innovative'
b'upcoming'
b'writer'
b'performed'
b'films'
b'arms'
b'married'
b'prosecutors'
b'agents'
b'handle'
b'duty'
b'captain'
b'securities'
b'employment'
b'surprised'
b'worried'
b'ninth'
b'pointed'
b'magazine'
b'threatened'
b'headquarters'
b'participants'
b'visiting'
b'press_release'
b'fly'
b'primarily'
b'respectively'
b'transportation'
b'younger'
b'jury'
b'respond'
b'rural'
b'acting'
b'factor'
b'update'
b'island'
b'flat'
b'remove'
b'photo'
b'mary'
b'selection'
b'scheme'
b'im

b'typical'
b'membership'
b'hiring'
b'consideration'
b'civilian'
b'fairly'
b'climate_change'
b'investigate'
b'violations'
b'pipeline'
b'seeks'
b'crore'
b'hire'
b'theme'
b'stressed'
b'resort'
b'walks'
b'roles'
b'rebound'
b'reference'
b'specialist'
b'timing'
b'imagine'
b'rapidly'
b'angry'
b'riding'
b'violation'
b'attitude'
b'guest'
b'select'
b'tech'
b'barrels'
b'sensitive'
b'hosting'
b'demanded'
b'marijuana'
b'intent'
b'strengthen'
b'channels'
b'ministers'
b'newspapers'
b'referred'
b'testified'
b'column'
b'arrive'
b'rushed'
b'kilometers'
b'frame'
b'provision'
b'authorized'
b'fail'
b'completion'
b'traveled'
b'heavily'
b'acquire'
b'striking'
b'virus'
b'secured'
b'breaks'
b'attractive'
b'indeed'
b'approve'
b'survived'
b'intense'
b'somewhat'
b'aims'
b'crews'
b'protected'
b'checks'
b'distributed'
b'talented'
b'kicked'
b'permission'
b'judgment'
b'beer'
b'clothes'
b'coal'
b'packed'
b'promoting'
b'flood'
b'approached'
b'towns'
b'disappointing'
b'collaboration'
b'hopefully'
b'streak'
b'balls'
b'pl

b'install'
b'affecting'
b'rugby'
b'forest'
b'cloud'
b'contractor'
b'chest'
b'signature'
b'uniform'
b'accidents'
b'intervention'
b'tougher'
b'teenager'
b'gambling'
b'mortgages'
b'standings'
b'evaluation'
b'planet'
b'dressed'
b'underground'
b'exact'
b'conservation'
b'midfielder'
b'depend'
b'replacing'
b'headline'
b'questioning'
b'athlete'
b'wide_range'
b'comparable'
b'accomplished'
b'ordinance'
b'feared'
b'discussing'
b'riders'
b'copper'
b'unexpected'
b'administrators'
b'favorable'
b'painting'
b'comply'
b'headquartered'
b'gross'
b'picks'
b'diplomatic'
b'lighting'
b'tobacco'
b'feelings'
b'pays'
b'documentary'
b'addresses'
b'senators'
b'linebacker'
b'cleaning'
b'withdrawal'
b'brilliant'
b'custom'
b'priorities'
b'economist'
b'waited'
b'furniture'
b'sub'
b'turnover'
b'costly'
b'height'
b'volumes'
b'till'
b'grab'
b'completing'
b'sample'
b'equipped'
b'delegates'
b'favorites'
b'shortage'
b'publisher'
b'estate'
b'hitter'
b'globe'
b'evaluate'
b'sits'
b'venture'
b'shoppers'
b'designated'
b'tying'


...

b'bolsters'
b'semesters'
b'grinned'
b'sprinkling'
b'composites'
b'mourns'
b'underestimating'
b'unwieldy'
b'funnier'
b'recouped'
b'populate'
b'pays_tribute'
b'migraines'
b'sodas'
b'bright_yellow'
b'annulled'
b'cube'
b'communicator'
b'bandh'
b'blue_ribbon'
b'heartwarming'
b'overzealous'
b'pork_barrel'
b'monopolies'
b'price_gouging'
b'pronouncement'
b'dislikes'
b'waterfalls'
b'flashback'
b'hunk'
b'engagement_ring'
b'drug_addict'
b'shivering'
b'scoops'
b'racists'
b'consolidates'
b'promos'
b'accommodates'
b'tidbits'
b'trims'
b'rollers'
b'smug'
b'dislodge'
b'outsized'
b'woe'
b'bookmarks'
b'devout'
b'shortly_afterwards'
b'active_ingredient'
b'localization'
b'metric_ton'
b'white_supremacist'
b'ghastly'
b'widowed'
b'siphoned'
b'uncooperative'
b'concession_stand'
b'surveyors'
b'scribe'
b'attribution'
b'drag_racing'
b'biographical'
b'bemoaned'
b'slowest_pace'
b'plastic_surgeon'
b'nicked'
b'mayoral_candidate'
b'piers'
b'incapacitated'
b'stomachs'
b'mashed_potatoes'
b'farm_subsidies'
b'feline'
b'ba

b'slur'
b'repelled'
b'hemp'
b'mercenary'
b'chiropractor'
b'disseminating'
b'decoy'
b'microcosm'
b'sinkhole'
b'committing_suicide'
b'flounder'
b'clinician'
b'grossly'
b'symbolically'
b'sprouting'
b'broken_ribs'
b'chuckling'
b'halfcourt'
b'stalker'
b'ops'
b'forested'
b'jostling'
b'allocates'
b'midyear'
b'navigated'
b'largesse'
b'swanky'
b'vested_interests'
b'authoring'
b'empirical'
b'pooch'
b'ballparks'
b'subtlety'
b'gusher'
b'renewable_fuels'
b'rebuked'
b'primary_endpoint'
b'descendant'
b'mausoleum'
b'sixties'
b'atypical'
b'massacred'
b'actively_engaged'
b'friendlier'
b'pigeon'
b'indecision'
b'giggle'
b'harmonize'
b'email_newsletter'
b'livid'
b'divested'
b'nots'
b'kidney_transplant'
b'mentally_disabled'
b'vernacular'
b'bereavement'
b'conjecture'
b'merited'
b'spades'
b'microbial'
b'sustenance'
b'reformists'
b'multiculturalism'
b'optimizes'
b'ambushes'
b'parkland'
b'edict'
b'boe'
b'commonly_referred'
b'tuxedo'
b'kickoff_returns'
b'intranet'
b'inkling'
b'reinvested'
b'stitching'
b'tombs'
b

b'flagrant_offenders'
b'gobbled'
b'constructively'
b'rearing'
b'renown'
b'eaters'
b'sputtered'
b'bulldog'
b'evictions'
b'instigating'
b'pancake'
b'thronged'
b'markup'
b'vociferous'
b'blond_hair'
b'gambit'
b'anthropologist'
b'profane_comments'
b'epitomizes'
b'ceding'
b'biochemical'
b'friendliness'
b'lacerations'
b'meth_lab'
b'lengthening'
b'rebranded'
b'upholstery'
b'authenticate'
b'devoured'
b'lengthened'
b'painkiller'
b'smallpox'
b'hereditary'
b'quieted'
b'affectionately'
b'infielders'
b'decentralization'
b'downed_trees'
b'nominally'
b'regular_intervals'
b'boat_ramp'
b'midterm_election'
b'worth_mentioning'
b'erodes'
b'deeply_divided'
b'corralled'
b'setups'
b'playwrights'
b'inhibited'
b'afterlife'
b'shipyards'
b'ulcers'
b'greening'
b'reversals'
b'hibernation'
b'simulating'
b'watersheds'
b'ruckus'
b'plucky'
b'raccoon'
b'detectable'
b'watcher'
b'authenticated'
b'misfired'
b'contrarian'
b'horrid'
b'reassert'
b'subsistence'
b'jumpstart'
b'minefield'
b'encoded'
b'focal'
b'dissenters'
b'gran

b'propagation'
b'rd'
b'chubby'
b'blowouts'
b'ferret'
b'wrangle'
b'tamoxifen'
b'rollicking'
b'cowards'
b'blue_skies'
b'uttering'
b'launder'
b'lemons'
b'minimally_invasive'
b'ness'
b'invigorating'
b'toolbox'
b'sofas'
b'micrograms'
b'lotion'
b'lawn_chairs'
b'dangled'
b'flaring'
b'giggles'
b'ridding'
b'wildest_dreams'
b'skirted'
b'greenfield'
b'unabashed'
b'oceanfront'
b'protruding'
b'kidney_stones'
b'orthodoxy'
b'athletic_trainer'
b'pluralism'
b'indistinguishable'
b'sighs'
b'hollering'
b'atrial_fibrillation'
b'sedate'
b'manly'
b'aspired'
b'recessed'
b'picket_lines'
b'vultures'
b'freezers'
b'customer_centric'
b'regimens'
b'shelf_registration'
b'helplessness'
b'co_ops'
b'latex'
b'cardholder'
b'screwing'
b'lynching'
b'delving'
b'cell_lung_cancer'
b'caskets'
b'populous'
b'blazer'
b'graphically'
b'clean_sheets'
b'superstition'
b'reconstituted'
b'umpiring'
b'repo_rate'
b'negotiable'
b'remedied'
b'trickier'
b'downpours'
b'unconvinced'
b'darting'
b'stringer'
b'ma'
b'salvo'
b'skim'
b'prepayment'
b

b'landlines'
b'soundly'
b'undercover_agent'
b'grins'
b'polysilicon'
b'bagels'
b'gazed'
b'commandeered'
b'frontage'
b'thrift_store'
b'oozing'
b'shakes_hands'
b'bare_minimum'
b'multimillionaire'
b'geometric'
b'reacted_angrily'
b'luscious'
b'figurines'
b'fraternal'
b'panache'
b'daylight_saving'
b'deplete'
b'thump'
b'interviewers'
b'metabolic_syndrome'
b'memorable_moments'
b'drapes'
b'knocked_unconscious'
b'touchy'
b'manifold'
b'psychotherapy'
b'finality'
b'headscarf'
b'stormy_weather'
b'outcast'
b'fining'
b'scuffled'
b'tactile'
b'tree_limbs'
b'trampling'
b'hallucinations'
b'deepwater_drilling'
b'urn'
b'noncommittal'
b'bestow'
b'inert'
b'subtext'
b'replenished'
b'anathema'
b'cleantech'
b'volts'
b'reevaluate'
b'bachelor_degrees'
b'ballerina'
b'overshadowing'
b'bumper_stickers'
b'uncharted_territory'
b'unenforceable'
b'forgoing'
b'tort_reform'
b'jailers'
b'exasperation'
b'conveyor_belt'
b'heparin'
b'pawns'
b'purveyor'
b'downwind'
b'allergens'
b'permeates'
b'membranes'
b'hijackings'
b'tribal_

In [9]:
wordsim.pprint_w2vnews(result_original, result_debiased)
# Question: which WS?

+---------------+----------+---------------+--+--+--+
|               | EN-RG-65 | EN-WS-353-ALL |  |  |  |
+---------------+----------+---------------+--+--+--+
|     Before    |   nan    |      nan      |
| Hard-debiased |   nan    |      nan      |
+---------------+----------+---------------+--+--+--+


# Full W2vNEWS set

## 1: original word embeddings on RG & WS

Sources:

#### RG: H. Rubenstein and J. B. Goodenough. Contextual correlates of synonymy. Communications of the ACM, 8(10):627–633, 1965.

####  WS: L. Finkelstein, E. Gabrilovich, Y. Matias, E. Rivlin, Z. Solan, G. Wolfman, and E. Ruppin. Placing search in context: The concept  revisited. In WWW. ACM, 2001.


In [10]:
# Load google news word2vec
E = WordEmbedding('./embeddings/GoogleNews-vectors-negative300.bin')

*** Reading data from ./embeddings/GoogleNews-vectors-negative300.bin
(3000000, 300)
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus


### Wordsim benchmarks
Code adapted from source 

#### embedding-evaluation: https://github.com/k-kawakami/embedding-evaluation

In [11]:
wordsim = Wordsim("en")
word2vec = wordsim.load_vector('./embeddings/GoogleNews-vectors-negative300.bin', binary=True)
result_original = wordsim.evaluate(word2vec)
wordsim.pprint(result_original)

load_vector
loaded vector 3000000 words found ..


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 2: Debiased word embeddings on RG & WS


### Step 2a: Hard debiased

In [12]:
from debiaswe.debias import debias

In [13]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
# print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]


In [14]:
debias(E, gender_specific_words, defs, equalize_pairs)
print("Saving to file...")
E.save('./embeddings/w2v_gnews_debiased_small.txt')
print("\n\nDone!\n")

3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus
{('gentlemen', 'ladies'), ('males', 'females'), ('FATHERHOOD', 'MOTHERHOOD'), ('FRATERNITY', 'SORORITY'), ('Father', 'Mother'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('dudes', 'gals'), ('KINGS', 'QUEENS'), ('dad', 'mom'), ('catholic_priest', 'nun'), ('boy', 'girl'), ('twin_brother', 'twin_sister'), ('CATHOLIC_PRIEST', 'NUN'), ('colt', 'filly'), ('COUNCILMAN', 'COUNCILWOMAN'), ('boys', 'girls'), ('King', 'Queen'), ('His', 'Her'), ('FATHERS', 'MOTHERS'), ('dads', 'moms'), ('prince', 'princess'), ('GELDING', 'MARE'), ('Spokesman', 'Spokeswoman'), ('Male', 'Female'), ('Wives', 'Husbands'), ('COLT', 'FILLY'), ('Brothers', 'Sisters'), ('Councilman', 'Councilwoman'), ('Fathers', 'Mothers'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('his', 'her'), ('grandson', 'granddaughter'), ('HE', 'SHE'), ('KING', 'QUEEN'), ('Businessman', 'Businesswoman'), ('Fella', 'Granny'), ('Testosterone', 'E

MemoryError: 

In [ ]:
# Do benchmark for hard-debiased
wordsim = Wordsim("en")
word2vec = wordsim.load_vector('./embeddings/w2v_gnews_debiased_small.txt')
result_debiased = wordsim.evaluate(word2vec)
print("ORIGINAL")
wordsim.pprint(result_original)
print("DEBIASED")
wordsim.pprint(result_debiased)

In [ ]:
wordsim.pprint_w2vnews(result_original, result_debiased)
# Question: which WS?